In [1]:
# Import
import math
import numpy as np
import pandas as pd
from gurobipy import Model, GRB, quicksum


# Dataの定義
IncomeData = np.array([
    [20, 10000, 3, "Commerce", 0],
    [30, 15500, 2, "Agriculture", 1000], 
    [25, 20000, 5, "Agriculture", 1000], 
    [18, 25000, 4, "Commerce", 3000],
    [32, 15000, 3, "Commerce", 500]
    ])

PopulationData = np.array([
    [25, 15000, 4, 40, 12, "M", 2],
    [30, 15000, 2, 25, 16, "M", 0],
    [18, 20000, 1, 30, 18, "F", 0],
    [27, 25000, 2, 35, 16, "F", 1],
    [25, 20000, 4, 25, 12, "M", 1],
    ])

model = Model()

## No. of Families
N_i = IncomeData[:, 0].astype(int)
N_j = PopulationData[:, 0].astype(int)

## Gross Family Income
G_i = IncomeData[:, 1].astype(int)
G_j = PopulationData[:, 1].astype(int)

## No. of Family Members
M_i = IncomeData[:, 2].astype(int)
M_j = PopulationData[:, 2].astype(int)

## 分散
s_G = np.var(np.concatenate([G_i, G_j])) # Gross
s_M = np.var(np.concatenate([M_i, M_j])) # Member
    
S = [(0, 0), (4, 0), (4, 1)]
# x_ij = [[20, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [5, 5, 0, 0, 0]]
x_ij = {(i, j): model.addVar(vtype=GRB.INTEGER) for i in range(5) for j in range(5)}
d_ij = {(i, j): model.addVar(vtype=GRB.CONTINUOUS) for i in range(5) for j in range(5)}



Restricted license - for non-production use only - expires 2024-10-28


In [2]:

model.addConstrs((quicksum(x_ij[i, j] for j in range(5)) - N_i[i] == 0 for i in [e[0] for e in S]))

## Σx_ij = N_j
model.addConstrs((quicksum(x_ij[i, j] for i in range(5)) - N_j[j] == 0 for j in [e[1] for e in S]))

## x_ij >= 0
model.addConstrs((x_ij[i, j] >= 0 for j in [e[1] for e in S] for i in [e[0] for e in S]))

# d
model.addConstrs(d_ij[i, j] == math.sqrt((G_i[i]-G_j[j])**2/s_G+(M_i[i]-M_j[j])**2/s_M) for i in [e[0] for e in S] for j in [e[1] for e in S])
## No. of Families
# N_i = IncomeData[:, 0].astype(int)
# N_j = PopulationData[:, 0].astype(int)
# i_star = next((index for index, value in enumerate(N_i) if value > 0), None)
# j_star = next((index for index, value in enumerate(N_j) if value > 0), None)
model.update()

In [3]:
model.setObjective(quicksum(d_ij[i,j]*x_ij[i,j] for i, j in S),GRB.MINIMIZE); #目的関数の追加

model.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 24 rows, 50 columns and 48 nonzeros
Model fingerprint: 0x809b14b3
Model has 3 quadratic objective terms
Variable types: 25 continuous, 25 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e-01, 3e+01]
Found heuristic solution: objective 21.1288564
Presolve removed 24 rows and 50 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 0 21.1289 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


In [4]:
#最適値の出力
print("Opt.value=",model.objval)

for i in range(5): #最適解の出力
    for j in range(5):
        print("x_ij[",i,"][",j,"]=",x_ij[i,j].X)

Opt.value= 0.0
x_ij[ 0 ][ 0 ]= -0.0
x_ij[ 0 ][ 1 ]= -0.0
x_ij[ 0 ][ 2 ]= 20.0
x_ij[ 0 ][ 3 ]= -0.0
x_ij[ 0 ][ 4 ]= -0.0
x_ij[ 1 ][ 0 ]= 25.0
x_ij[ 1 ][ 1 ]= 30.0
x_ij[ 1 ][ 2 ]= -0.0
x_ij[ 1 ][ 3 ]= -0.0
x_ij[ 1 ][ 4 ]= -0.0
x_ij[ 2 ][ 0 ]= -0.0
x_ij[ 2 ][ 1 ]= -0.0
x_ij[ 2 ][ 2 ]= -0.0
x_ij[ 2 ][ 3 ]= -0.0
x_ij[ 2 ][ 4 ]= -0.0
x_ij[ 3 ][ 0 ]= -0.0
x_ij[ 3 ][ 1 ]= -0.0
x_ij[ 3 ][ 2 ]= -0.0
x_ij[ 3 ][ 3 ]= -0.0
x_ij[ 3 ][ 4 ]= -0.0
x_ij[ 4 ][ 0 ]= -0.0
x_ij[ 4 ][ 1 ]= -0.0
x_ij[ 4 ][ 2 ]= -0.0
x_ij[ 4 ][ 3 ]= -0.0
x_ij[ 4 ][ 4 ]= 32.0
